# Lime para texto


In [ ]:
# !pip install 'scikit-learn==0.24.1'

import sklearn
sklearn.__version__

### Imports e demais inicializações

In [ ]:
import lime
import warnings
import numpy as np
import pandas as pd
import cloudpickle
from lime import lime_text
import matplotlib.pyplot as plt
from lime.lime_text import LimeTextExplainer
from sklearn.feature_extraction.text import TfidfVectorizer

%run modules/data/text.py
%run modules/features/processingFeatures.py


warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
np.set_printoptions(suppress=True, precision=3)

### Carrega os dados

In [ ]:
validacao = pd.read_csv('dataset/validacao_interpretabilidade.csv')
validacao['paragrafos'] = validacao['paragrafos'].apply(cleaning)
print(validacao.shape)

### Carrega o modelo

In [ ]:
model = cloudpickle.load(open('model/predict_lime.sav', 'rb'))

# Interpretabilidade

In [ ]:
class_names = ['dano material (0)', 'dano moral (1)']
explainer = LimeTextExplainer(class_names=class_names)

## Gerando explicações para 1 instância

In [ ]:
idx = 100
texto = validacao['paragrafos'][idx]
exp = explainer.explain_instance(texto, model.predict_proba, num_features=6)
listaExplicacao = exp.as_list()

print('Documento id: %d' % idx)
print('Classe predita', class_names[model.predict([texto])[0]])
print('Classe real: %s' % validacao['categorias'][idx])
print('\nExplicação:')
print ('\n'.join(map(str, listaExplicacao)))

In [ ]:
print('Score(R²): ', exp.score)
exp.show_in_notebook(text=True)

## Avaliação das explicações

Lista gerado pelo Lime

In [ ]:
def getLimeList(listaExplicacao):
    lime_list = []
    for palavras in listaExplicacao:
        lime_list.append(palavras[0])
    return lime_list

Lista com todas as palavras do texto ordenadas por importaância de acordo com a pontuação TF-IDF

In [ ]:
vectorizer = TfidfVectorizer()

def generateTfidfList(texto):
    text_db = [cleaning(texto)]
    tf_idf =  vectorizer.fit_transform(text_db)
    result = pd.DataFrame(tf_idf.toarray(), columns=vectorizer.get_feature_names())
    result = result.T.sort_values(by=0, ascending=False)
    return result.index.values

### Anotação Humana x Lime

In [ ]:
res = {'lime_list': [], 'humano_list': [], 'categoria': []}
count = 0

for i in range(len(validacao)):
    count+=1    
    if count%10==0:
        print(f'Concluido etapa {count}')

    texto = validacao['paragrafos'][i]     
    exp = explainer.explain_instance(texto, model.predict_proba, num_features=6)
    lime_list = getLimeList(exp.as_list())
    res['lime_list'].append(lime_list)
    res['humano_list'].append(validacao['trechos_tokens'][i])
    res['categoria'].append(validacao['categorias'][i])
frame = pd.DataFrame(res)
frame.to_csv('result.csv', index=False)

Recall

In [ ]:
resultado = pd.read_csv('dataset/result_lime_humano.csv')
resultado['lime_list'] = resultado['lime_list'].apply(prepare)
resultado['humano_list'] = resultado['humano_list'].apply(prepare)

acerto = 0
scores = []

for i in range(len(resultado)):
    for lime in resultado['lime_list'][i]:
        if lime in resultado['humano_list'][i]:
            acerto+=1
    scores.append(acerto / len(resultado['humano_list'][i])*100)
    acerto = 0
print('% médio de acertos: ', np.mean(scores).round(2))

### Coeficiente de correlação de classificação de Kendall

> É uma medida de correspondência entre duas classificações. Valores próximos a 1 indicam forte concordância e valores próximos a -1 indicam forte discordância.

In [ ]:
import scipy.stats as stats

def calculateTauKendall(lime_list, tfidf_list):
    interim_list = []
    for word in tfidf_list:
        if word in lime_list and word not in interim_list:
            interim_list.append(word)
    tau, p_value = stats.kendalltau(lime_list, interim_list)
    print(interim_list)
    return tau, p_value

Gera a explicação para toda a base de validação e calcula a média para o tau de kendall e o p-value

In [ ]:
scores_tau = []
scores_rquadrado = []
p_values = []
for i in range(len(validacao)):
    texto = validacao['paragrafos'][i]     
    exp = explainer.explain_instance(texto, model.predict_proba, num_features=6)
    
    scores_rquadrado.append(exp.score)
    
    lime_list = getLimeList(exp.as_list())
    tfidf_list = generateTfidfList(texto)
    
    tau, p_value = calculateTauKendall(lime_list, tfidf_list)
    scores_tau.append(tau)
    p_values.append(p_value)

Analisando o p-value: 
> **H0**: As amostras não estão correlacionada

> **H1**: As amostras estão correlacionadas

In [ ]:
print('Tau para 500 exemplos: ', np.mean(scores_tau).round(3))
print('Score médio para 500 exemplos: ', np.mean(scores_rquadrado).round(3))

alpha = 0.05
p = np.mean(p_values)
if p > alpha:
    print('As amostras não são correlacionadas (falha em rejeitar H0) p=%.3f' % p)
else:
    print('As amostras são correlacionadas (rejeita H0) p=%.3f' % p)

### Score médio das explicações

In [ ]:
scores = []
for i in range(len(validacao)):
    texto = cleaning(validacao['paragrafos'][i])                     
    exp = explainer.explain_instance(texto, model.predict_proba, num_features=6)
    scores.append(exp.score)
print('Score médio para 500 exemplos: ', np.mean(scores).round(3))

### Precisão, revocação e F1

In [ ]:
def getMetrics(lime_list, tfidf_list):
    len_lime_list =  len(lime_list)
    len_tfidf_list = len(tfidf_list)
    total_palavras_tfidf_list_em_lime_list = 0

    for i in range(len_lime_list):
        if tfidf_list[i] in lime_list:
            total_palavras_tfidf_list_em_lime_list += 1

    sublist_lime_len = len_lime_list
    while total_palavras_tfidf_list_em_lime_list != len_lime_list:
        if tfidf_list[sublist_lime_len] in lime_list:
            total_palavras_tfidf_list_em_lime_list += 1
        sublist_lime_len += 1

    recall = sublist_lime_len / len_tfidf_list
    precision = len_lime_list / sublist_lime_len
    f1_score = 2 * precision * recall / (precision + recall)
    return recall, precision, f1_score

In [ ]:
recall, precision, f1_score = getMetrics(lime_list, tfidf_list)
print(f' Precisão: {precision}')
print(f' Recall: {recall}')
print(f'F1-escore: {f1_score}')